<a href="https://colab.research.google.com/github/eduglez03/RedesNeuronales/blob/main/RedesNeuronalesConvolucionales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Práctica con Redes Neuronales Convolucionales usando Keras

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
2.18.0


## Datos de MNIST

In [2]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D

# Función de activación "Relu", tamaño de la imagen "input_shape=(28, 28, 1)", el 1 es el canal de color
model = Sequential()
# Usamos una vantana de 5x5 en la capa de convolución
model.add(Conv2D(32, (5, 5), activation='relu', input_shape=(28, 28, 1)))
# Usamos un MaxPooling con una ventana de 2x2 para la capa de pooling
model.add(MaxPooling2D((2, 2)))
model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 24, 24, 32)          │             832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 12, 12, 32)          │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 832 (3.25 KB)

 Trainable params: 832 (3.25 KB)

 Non-trainable params: 0 (0.00 B)

#Implementación de la red

In [3]:
model = Sequential()
model.add(Conv2D(32, (5, 5), activation='relu', input_shape=(28, 28, 1)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (5, 5), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_1 (Conv2D)                    │ (None, 24, 24, 32)          │             832 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 12, 12, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 8, 8, 64)            │          51,264 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 4, 4, 64)            │               0 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 52,096 (203.50 KB)

 Trainable params: 52,096 (203.50 KB)

 Non-trainable params: 0 (0.00 B)

In [4]:
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten

# Añadimos a la red una capa densa que servirá para alimentar una capa final de "softmax", para hacer clasificación
model.add(Flatten()) # El método "Flatten" de keras nos permite aplanar los vectores 3D a un vector 1D para poder pasarselo a la Softmax
model.add(Dense(10, activation='softmax'))

##Procesamiento de los datos

In [5]:
# Las Redes Neuronales Convolucionales esperan tensores 3D como entradas por lo que convertimos las imagenes en 3D, donde en este caso el tercer canal es un 1 puesto
# que las imagenes son en blanco y negro
from keras.utils import to_categorical

mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

print (train_images.shape)
print (train_labels.shape)
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

print (train_images.shape)
print (train_labels.shape)

11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
(60000, 28, 28)
(60000,)
(60000, 28, 28, 1)
(60000, 10)


##Configuración de entrenamiento y entrenamiento

In [6]:
# Configuramos el entrenamiento con función de pérdida y optimizador
model.compile(loss='categorical_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

# Entrenamos el modelo
model.fit(train_images, train_labels,
          batch_size=100,
          epochs=5,
          verbose=1)

# Evaluamos el modelo
test_loss, test_acc = model.evaluate(test_images, test_labels)

print('Test accuracy:', test_acc)

Epoch 1/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 62s 103ms/step - accuracy: 0.5813 - loss: 1.5233
Epoch 2/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 61s 101ms/step - accuracy: 0.9172 - loss: 0.2868
Epoch 3/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 82s 101ms/step - accuracy: 0.9414 - loss: 0.1991
Epoch 4/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 81s 98ms/step - accuracy: 0.9545 - loss: 0.1576
Epoch 5/5
600/600 ━━━━━━━━━━━━━━━━━━━━ 81s 97ms/step - accuracy: 0.9622 - loss: 0.1309
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 15ms/step - accuracy: 0.9638 - loss: 0.1235
Test accuracy: 0.9674000144004822
